In [ ]:
%pip install langgraph langchain

In [5]:
from langgraph.func import entrypoint, task
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

@task
def initial_screening(resume: str) -> dict:
    print("--- TASK: AI doing initial screening ---")
    return {"score": 85, "recommendation": "Strong candidate"}

@task
def technical_test(candidate_id: str) -> dict:
    print(f"--- TASK: {candidate_id} running technical test ---")
    return {"test_score": 90, "passed": True}

@entrypoint(checkpointer=MemorySaver())
def multi_approval_workflow(candidate_data: dict) -> str:
    candidate_name = candidate_data["name"]
    resume = candidate_data["resume"]

    print(f"\n--- WORKFLOW: Processing candidate: {candidate_name} ---")

    screening = initial_screening(resume)

    print("--- WORKFLOW: PAUSE 1 - Awaiting HR approval ---")
    hr_approval = interrupt("HR: Approve initial screening? (yes/no)")

    if hr_approval != "yes":
        return f"REJECTED at HR screening for {candidate_name}"

    print(f"--- WORKFLOW: HR approved {candidate_name}. Proceeding. ---")

    tech_result = technical_test(candidate_name)

    print("--- WORKFLOW: PAUSE 2 - Awaiting Manager approval ---")
    manager_approval = interrupt("Manager: Approve for hire? (yes/no)")

    if manager_approval != "yes":
        return f"REJECTED by Manager for {candidate_name}"

    print(f"--- WORKFLOW: Manager approved {candidate_name}. ---")

    return f"HIRED - {candidate_name}"

if __name__ == "__main__":
    thread_config = {"thread_id": "jane_002"}
    candidate_payload = {"name": "Jane Smith", "resume": "Python expert"}

    print("="*60)
    print("RUN 1: To first interrupt (HR Approval)")
    print("="*60)
    multi_approval_workflow.invoke(candidate_payload, config=thread_config)

    print("\n" + "="*60)
    print("RUN 2: HR approves, to second interrupt (Manager Approval)")
    print("="*60)
    multi_approval_workflow.invoke(None, config=thread_config, resume=Command(resume="yes"))

    print("\n" + "="*60)
    print("RUN 3: Manager approves, to completion")
    print("="*60)
    final_status = multi_approval_workflow.invoke(None, config=thread_config, resume=Command(resume="yes"))

    print(f"\n--- FINAL STATUS ---")
    print(final_status)

RUN 1: To first interrupt (HR Approval)

--- WORKFLOW: Processing candidate: Jane Smith ---
--- WORKFLOW: PAUSE 1 - Awaiting HR approval ---
--- TASK: AI doing initial screening ---

RUN 2: HR approves, to second interrupt (Manager Approval)

--- WORKFLOW: Processing candidate: Jane Smith ---
--- WORKFLOW: PAUSE 1 - Awaiting HR approval ---

RUN 3: Manager approves, to completion

--- WORKFLOW: Processing candidate: Jane Smith ---
--- WORKFLOW: PAUSE 1 - Awaiting HR approval ---

--- FINAL STATUS ---
{'__interrupt__': [Interrupt(value='HR: Approve initial screening? (yes/no)', id='5d5d6eda05d5c3dd16f556ab0ddedaaa')]}
